In [2]:
from collections import namedtuple
import os
os.chdir("../")  ## changing the directory to root folder

In [3]:
os.getcwd()

'f:\\Machine_learning-End to end projects\\CNN_Deep_learning_project'

In [4]:
DataIngestionConfig = namedtuple("DataIngestionConfig",[
    "root_dir",
    "source_URL",
    "local_data_file",
    "unzip_dir"
])

In [5]:
from dataclasses import dataclass  #This will help us create same structure as above namedtuple
from pathlib import Path


@dataclass(frozen=True)  ## 
class DataIngestionConfig:
    root_dir: Path   ## here type will be path
    source_URL: str     ## here type will be 
    local_data_file: Path
    unzip_dir: Path

In [6]:
from DeepClassifier.constants import *
from DeepClassifier.utils import read_yaml,create_directories

In [7]:
from distutils.command.config import config


class ConfigurationManager:
    def __init__(self,
    config_filepath = CONFIG_FILE_PATH,
    params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion 

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL=config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

        

In [8]:
import os
import urllib.request as request   ## for downloading the data its the kibrary
from zipfile import ZipFile         ## for zipping  and unzipping the file data

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):  ## To check if file is present or not
            filename, headers = request.urlretrieve(        
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )

    def _get_updated_list_of_files(self, list_of_files):  ## To get the list of files that ends with extension .jpg and inages of cat and dog
        return [f for f in list_of_files if f.endswith(".jpg") and ("Cat" in f or "Dog" in f)]

    def _preprocess(self, zf: ZipFile, f: str, working_dir: str):
        target_filepath = os.path.join(working_dir, f)  
        if not os.path.exists(target_filepath):
            zf.extract(f, working_dir)
        
        if os.path.getsize(target_filepath) == 0:  ## To check if the file is having the size 0
            os.remove(target_filepath)

    def unzip_and_clean(self):    ## To unzip the file that we want to open
        with ZipFile(file=self.config.local_data_file, mode="r") as zf:   ##zf is zip file alias
            list_of_files = zf.namelist()    ## To get name list of files
            updated_list_of_files = self._get_updated_list_of_files(list_of_files)  ## 
            for f in updated_list_of_files:
                self._preprocess(zf, f, self.config.unzip_dir)    ## f is the pointer to updated_list_of_files
                                                                ##
                                                                                

In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_and_clean()
except Exception as e:
    raise e

ValueError: yaml file is empty